Assignment #1: Regression
----

<center><img src="https://i.imgur.com/s888wFw.png" width="75%"/></center>

Learning Outcomes
------

__By the end of this assignment, you should be able to__:

- Practice applying the Data Science worklflow to a real-world problem.
- Fit a regression model with scikit-learn.

Data Science Workflow:
======

1. Ask
2. Acquire
3. Process
4. Model
5. Deliver 

1) Ask: 
----

Can a linear regression model learn to predict the price of airbnb rentals in San Francisco?

2) Acquire
----

The data was sourced from [airbnb](https://www.airbnb.com/) and converted to a .csv for you.

In [29]:
import numpy as np
import pandas as pd

In [30]:
# path = "../../data/regression/"
path = "./"

# Load train data - train data has potential features and price as target
data_train_raw = pd.read_csv(path+"train.csv", low_memory=False)
assert "price" in data_train_raw.columns

target_raw = data_train_raw.pop('price')
# Convert `price` as a string with dollar signs and commas to a float.
y_train = np.array(target_raw.str.replace('$', '').str.replace(',', '').astype(float))

# Keep track of the number of instances
n_rows_input = data_train_raw.shape[0]

/var/folders/jp/_tvsjv2j23q4f6wnb__01z280000gn/T/ipykernel_42946/2872267030.py:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  y_train = np.array(target_raw.str.replace('$', '').str.replace(',', '').astype(float))


3) Process
-----

In [31]:
def process_data(data):
    """Feature engineering & feature selection
    The input parameter and return value have the same name. All processing should happen inplace.
    """

    ### BEGIN SOLUTION

    # Replace with NaN with median (because of skewed data)
    data['reviews_per_month'].fillna(data['reviews_per_month'].median(), inplace=True)
    data['review_scores_rating'].fillna(data['review_scores_rating'].median(), inplace=True)
    data['host_response_rate'] = data['host_response_rate'].str.replace('%', '').astype(float)
    data['host_response_rate'].fillna(data['host_response_rate'].median(), inplace=True)
    # data['cleaning_fee'] = data['cleaning_fee'].str.replace('$', '').astype(float)
    # data['cleaning_fee'].fillna(data['cleaning_fee'].median(), inplace=True)
    # data['security_deposit'] = data["security_deposit"].str.replace('$', '').str.replace(',', '').astype(float)
    # data['security_deposit'].fillna(data['security_deposit'].median(), inplace=True)
    
    cols_to_keep = [
                    # 'cleaning_fee',
                    'host_response_rate',
                    # 'security_deposit', 
                    'number_of_reviews',
                    'maximum_nights', 
                    'minimum_nights',
                    'reviews_per_month',
                    'review_scores_rating']

    data = data[cols_to_keep]
    
    ### END SOLUTION
    
    return data

In [32]:
"""
2 points
Test code for the 'process_data' function. 
This cell should NOT give any errors when it is run.
"""

data_train = process_data(data_train_raw)

# Double check the type is still a pd.DataFrame
assert type(data_train) == pd.core.frame.DataFrame
# Double check no rows where drop. Dropping which will break test data performance.
assert data_train.shape[0] == n_rows_input 

In [33]:
# Convert from pandas.DataFrame (tabular) to numpy.array (matrix)
X_train = np.array(data_train)

4) Model
-----

Only use scikit-learn's [Generalized Linear Models (GLM)](https://scikit-learn.org/stable/modules/linear_model.html). No other models are allowed.

No automatic hyperparameter search is allowed in final submitted code, thus no model with `CV` suffix. 

In [34]:
from sklearn import linear_model

In [35]:
# Fit simple LR
lm = linear_model.LinearRegression() # TODO: Replace with your choice of algorithm and hyperparameters 
lm.fit(X_train, y_train) # Train model

### BEGIN SOLUTION

lm =  linear_model.HuberRegressor(                # Choose Huber Regression because it is robust to outliers.
                                    epsilon=2,    # Increase epsilon so outliers have less impact.
                                    max_iter=50,  # Decrease training time increases training performance.
                                    alpha=.1)     # Increase regularization to increase chance of generalization.
# Note that X values are not normalized. Will discuss why on feature engineering day.
lm.fit(X_train, y_train)

### END SOLUTION

/Users/brian/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_huber.py:342: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


HuberRegressor(alpha=0.1, epsilon=2, max_iter=50)

In [36]:
"""
1 point
Test for valid model type.
This cell should NOT give any errors when it is run.
"""

assert str(type(lm)).split(".")[1] == 'linear_model'
assert str(type(lm)).split(".")[-1].count('CV') == 0

Model Evaluation
------

__Root Mean Squared Logarithmic Error__

$$ RMSLE = \sqrt{\frac{1}{n}\sum_{i=1}(log(p_i + 1) - log(a_i + 1))^2}$$

p = predicted

a = actual

In [37]:
def rmlse(target_true, target_predicted):
    "Root Mean Squared Logarithmic Error"
    
    assert len(target_true) == len(target_predicted), "True and predicted targets need to be the same length"

    log_diff = np.log1p(target_predicted + 1) - np.log1p(target_true + 1)
    return np.sqrt(np.mean(np.power(log_diff, 2)))

In [38]:
rmlse_value = rmlse(lm.predict(X_train), y_train)
print(f"{rmlse_value:.4f} rmlse on training set")

0.6576 rmlse on training set


__Median Absolute Error__

$$\text{MedAE} = \text{median}(\mid p_1 - a_1 \mid, \ldots, \mid p_n - a_n \mid)$$

In [39]:
from sklearn import metrics

medae_value = metrics.median_absolute_error(y_train, lm.predict(X_train))
print(f"{medae_value:.4f} medae on training set")

58.7755 medae on training set


5) Deliver
-----

How well can your model predict the test data?

In [40]:
# Load test data - test data only has potential features, no price as target
data_test_raw = pd.read_csv(path+"test.csv", low_memory=False)
assert not "price" in data_test_raw.columns

n_rows_input = data_test_raw.shape[0]

In [41]:
"""
2 points
Test code for the 'process_data' function. 
This cell should NOT give any errors when it is run.
"""

data_test = process_data(data_test_raw)

# Double check the type is still a pd.DataFrame
assert type(data_test) == pd.core.frame.DataFrame

# Double check not dropping any rows which will break test data
assert data_test.shape[0] == n_rows_input 

In [42]:
# Performance on median benchmark (mock test dataset)
# Sanity-check that your model can generate predicts in the correct form for actual test dataset.
# Your performance on the median data is not a good indicator of preformance on actual test dataset.

X_test = np.array(data_test)

test_solutions = pd.read_csv(path+'median_benchmark.csv') 
y_median = test_solutions['median_price']

medae_value = metrics.median_absolute_error(y_median, lm.predict(X_test))
print(f"{medae_value:.4f} medae on median test set")

13.4398 medae on median test set


In [43]:
"""
1 point for medae less than 100 on test dataset.
Tests are hidden since you don't have access to test targets.
"""

### BEGIN HIDDEN TESTS

# # Performance on test set
# test_solutions = pd.read_csv(path+'/instructor/test_solutions.csv')
# y_test = test_solutions['price']
# medae_value = metrics.median_absolute_error(y_test, lm.predict(X_test))
# print(f"{medae_value:.4f} medae on test set")

# assert medae_value < 100
#### END HIDDEN TESTS 


"\n1 point for medae less than 100 on test dataset.\nTests are hidden since you don't have access to test targets.\n"

In [44]:
"""
1 point for medae less than 90 on test dataset.
Tests are hidden since you don't have access to test targets.
"""

### BEGIN HIDDEN TESTS
assert medae_value < 90
#### END HIDDEN TESTS 


In [45]:
"""
1 point for medae less than 80 on test dataset.
Tests are hidden since you don't have access to test targets.
"""

### BEGIN HIDDEN TESTS
assert medae_value < 80
#### END HIDDEN TESTS 


In [46]:
"""
1 point for medae less than 70 on test dataset.
Tests are hidden since you don't have access to test targets.
"""

### BEGIN HIDDEN TESTS
assert medae_value < 70
#### END HIDDEN TESTS 


In [47]:
"""
1 point for medae less than 60 on test dataset.
Tests are hidden since you don't have access to test targets.
"""

### BEGIN HIDDEN TESTS
assert medae_value < 60
#### END HIDDEN TESTS 


Helpful Hints
------

- The primary deliverables are `process_data` function and a single linear model, aka `lm`. There should be no other code in the notebook, including visualizations or hyperparameter search code.
- Suggested steps (deliverable / nondeliverable)
    1. Exploratory data analysis (EDA) (nondeliverable)
    2. Process data to create features (deliverable)
    3. Search for model, hyperparamter, parameters (nondeliverable)
    4. Select single algorithm with single set of hyperparamter (deliverable)
- There are __no__ tricks to doing well on the test dataset. This assignment requires fundamental machine learning skills:
    - Domain and data knowledge
    - Feature engineering and feature selection
    - Model selection and hyperparameter tuning
- Domain knowledge is always important - Check out [airbnb San Francisco listings](https://www.airbnb.com/a/San-Francisco--California--United-States) to understand the data.
- Use your EDA skills. Create a separate notebook for EDA.
- Clean data is better than dirty data.
- Think deeply about the data. What potential issues will arise with modeling this data with a linear regression algorithm?
- High quality regression results often require hand-tuning of features and model hyperparameters.
- Academic Integrity reminder - Use only course notes and package (scikit-learn, pandas, …) documentation for this assignment. You should __not__ be googling for help. You should be working independently.

<br>
<br> 
<br>

----